In [1]:
import pytorch_lightning as pl
import torch_geometric
import wandb
from pytorch_lightning.loggers import WandbLogger
import time
from smtr_e3nn_model import SMTR, create_transform, Ligand_dataset

/home/huabei/anaconda3/envs/smtr2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
learning_rate = 1e-5
max_epoch = 5
if_new_train = True
check_point = '20220516_114821.ckpt'


In [11]:
project = 'smtr_e3nn'
wandb.init(project=project)
wandb_logger = WandbLogger(save_dir='.')
train_dataset = '../dataset/data_train.txt'
val_dataset = '../dataset/data_test.txt'
transform = create_transform()


/home/huabei/anaconda3/envs/smtr2/lib/python3.8/site-packages/pytorch_lightning/loggers/wandb.py:347: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(


In [12]:

# data
train_dataset = Ligand_dataset(train_dataset, transform=transform)
val_dataset = Ligand_dataset(val_dataset, transform=transform)
train_dataloader = torch_geometric.loader.DataLoader(train_dataset, batch_size=8)
val_dataloader = torch_geometric.loader.DataLoader(val_dataset, batch_size=8)



In [13]:
if if_new_train:
    # file_position = time.strftime("%Y%m%d_%H%M%S", time.localtime())
    # os.mkdir(file_position)
    smnn = SMTR(learning_rate=learning_rate)
else:
    smnn = SMTR().load_from_checkpoint(checkpoint_path=check_point)
trainer = pl.Trainer(logger=wandb_logger, max_epochs=max_epoch, accumulate_grad_batches=2, terminate_on_nan=True,
                        auto_scale_batch_size=True, accelerator='gpu', devices=1)



/home/huabei/anaconda3/envs/smtr2/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:569: LightningDeprecationWarning: Trainer argument `terminate_on_nan` was deprecated in v1.5 and will be removed in 1.7. Please use `Trainer(detect_anomaly=True)` instead.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [14]:
trainer.fit(smnn, train_dataloader, val_dataloader)
trainer.save_checkpoint(time.strftime("%Y%m%d_%H%M%S", time.localtime()) + ".ckpt")
wandb.finish()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name     | Type           | Params
---------------------------------------------
0  | elu      | ELU            | 0     
1  | lin1     | Linear         | 192   
2  | conv10   | Convolution    | 7.5 K 
3  | conv11   | Convolution    | 7.0 K 
4  | conv12   | Convolution    | 7.0 K 
5  | norm10   | Norm           | 0     
6  | norm11   | Norm           | 0     
7  | norm12   | Norm           | 0     
8  | lin20    | Linear         | 576   
9  | lin21    | Linear         | 576   
10 | lin22    | Linear         | 576   
11 | nonlin10 | NormActivation | 24    
12 | nonlin11 | NormActivation | 24    
13 | nonlin12 | NormActivation | 24    
14 | lin30    | Linear         | 288   
15 | lin31    | Linear         | 288   
16 | lin32    | Linear         | 288   
17 | conv2    | ModuleDict     | 27.8 K
18 | norm20   | Norm           | 0     
19 | norm21   | Norm           | 0     
20 | norm22   | Norm           | 0     
21 | lin40    | Linear         

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/huabei/anaconda3/envs/smtr2/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home/huabei/anaconda3/envs/smtr2/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 191. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/home/huabei/anaconda3/envs/smtr2/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 139. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/home/huabei/anaconda3/envs/smtr2/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve perform

Epoch 0:  29%|██▉       | 3920/13486 [26:21<1:04:20,  2.48it/s, loss=3.18, v_num=y1t9] 

ValueError: The loss returned in `training_step` is nan.

epoch,▁▁▁▁▁
train_loss,▇▇█▂▁
trainer/global_step,▁▃▅▆█
epoch,0
train_loss,7.30032
trainer/global_step,249
